In [ ]:
from typing import List, Tuple
import itertools
import math
import time
import PIL.Image
import PIL.ImageDraw
import PIL.ImageFont
from IPython.display import HTML
from tqdm import tqdm

In [ ]:
def reduce_edges(edges: List[Tuple[int, ...]]) -> List[Tuple[int, ...]]:
    """Remove symmetries and rotations from triangle edge indices"""
    num_items = len(edges[0])
    edge_length = num_items//3 + 1
    result_set = set()
    reduced_edges = set()
    def _reduce_to_3(edges):
        if edge_length < 4:
            return edges
        # make eg. (1, sum(2, 3), 4, sum(5, 6), 7, sum(8, 9))
        return (
            edges[0],
            sum(edges[1: edge_length - 1]),
            edges[edge_length - 1],
            sum(edges[edge_length: edge_length * 2 - 2]),
            edges[edge_length * 2 - 2],
            sum(edges[-edge_length + 2:]),
        )
        
    for edge_org in sorted(edges):
        edge = _reduce_to_3(edge_org)
        found = False
        for i in range(2):
            if edge in reduced_edges:
                found = True
                break
            for j in range(2):
                edge = edge[3 - 1:] + edge[:3 - 1]
                if edge in reduced_edges:
                    found = True
                    break
            edge = tuple(reversed(edge))
            edge = edge[-1:] + edge[:-1]
        
        if not found:
            reduced_edges.add(edge)
            result_set.add(edge_org)
    return sorted(result_set)
    
reduce_edges(
    #[(4, 2, 6, 1, 5, 3), (4, 3, 5, 1, 6, 2), (5, 1, 6, 2, 4, 3), (5, 3, 4, 2, 6, 1), (6, 1, 5, 3, 4, 2), (6, 2, 4, 3, 5, 1)]
    #[(1, 4, 9, 3, 5, 7, 2, 6, 8), (1, 4, 9, 3, 5, 7, 2, 8, 6), (1, 4, 9, 3, 7, 5, 2, 6, 8), (1, 4, 9, 3, 7, 5, 2, 8, 6), (1, 5, 9, 2, 4, 8, 3, 6, 7), (1, 5, 9, 2, 4, 8, 3, 7, 6), (1, 5, 9, 2, 8, 4, 3, 6, 7), (1, 5, 9, 2, 8, 4, 3, 7, 6), (1, 6, 7, 3, 4, 8, 2, 5, 9), (1, 6, 7, 3, 8, 4, 2, 5, 9), (1, 6, 8, 2, 5, 7, 3, 4, 9), (1, 6, 8, 2, 7, 5, 3, 4, 9), (1, 7, 6, 3, 4, 8, 2, 5, 9), (1, 7, 6, 3, 8, 4, 2, 5, 9), (1, 8, 6, 2, 5, 7, 3, 4, 9), (1, 8, 6, 2, 7, 5, 3, 4, 9)]
    [(10, 1, 5, 8, 12, 2, 4, 8, 11, 3, 6, 7), (10, 1, 5, 8, 12, 2, 4, 8, 11, 6, 3, 7)]
)

In [ ]:
def find_magic_triangles(
        edge_length: int = 3,
        start_offset: int = 1,
        scale: int = 50,
        exponent: int = 1,
        verbose: bool = True,
):
    num_items = (edge_length - 1) * 3
    display(HTML(f"""<h2>EDGE LENGTH: {edge_length}, ITEMS: {num_items}! = {math.factorial(num_items):,}</h2>"""))
    magics = {}
    try:
        for indices in tqdm(itertools.permutations([
                (i + start_offset) ** exponent
                for i in range(num_items)
        ]), total=math.factorial(num_items), disable=not verbose):
            edge1 = indices[:edge_length]
            edge2 = indices[edge_length - 1: edge_length * 2 - 1]
            sum1 = sum(edge1)
            sum2 = sum(edge2)
            if sum2 != sum1:
                continue
            edge3 = indices[edge_length * 2 - 2:] + indices[:1]
            sum3 = sum(edge3)
            if sum3 != sum1:
                continue
            magics.setdefault(sum1, []).append(indices)
    except KeyboardInterrupt:
        print("INTERRUPTED!")
        pass
    #magics = {0: [(1, 2, 3, 4, 5, 6)]}
    if verbose:
        time.sleep(1)
    for key in sorted(magics):
        edges = reduce_edges(magics[key])
        corner_sums = set()
        for edge in edges:
            corner_sums.add(edge[0] + edge[edge_length-1] + edge[-edge_length+1])
        print(f"SUM: {key}, VARIATIONS: {len(edges)}, CORNER SUM: {', '.join(str(s) for s in sorted(corner_sums))}")

        disp_edges = edges[:20]
        image = PIL.Image.new("L", (scale * 2 * len(disp_edges), scale), 0)
        draw = PIL.ImageDraw.ImageDraw(image)
        for j, edge in enumerate(disp_edges):
            for i, e in enumerate(edge):
                if i < edge_length:
                    tx, ty = i, i
                elif i < edge_length * 2 - 1:
                    tx, ty = i, edge_length * 2 - i - 2
                else:
                    tx, ty = (edge_length - (i-edge_length*2)*2 - max(0, 6-edge_length)), 0
                draw.text((j * scale * 2 + tx * scale / edge_length, scale - 12 - ty * scale / edge_length), str(e), fill=255)
                    
        display(image)
find_magic_triangles(3, exponent=1, start_offset=1)

# Summing inner edges

## Solutions to edge-length 3 (720)

### Exponent 1

    SUM=9, VARIATIONS=1
    SUM=10, VARIATIONS=1
    SUM=11, VARIATIONS=1
    SUM=12, VARIATIONS=1

### Exponent 2

    None
    
## Solutions to edge-length 4 (362,880)

### Exponent 1

    SUM=17, VARIATIONS=1
    SUM=19, VARIATIONS=2
    SUM=20, VARIATIONS=4
    SUM=21, VARIATIONS=2
    SUM=23, VARIATIONS=1

### Exponent 2

    SUM=126, VARIATIONS=1
    
## Solutions to edge-length 5 (479,001,600) 

### Exponent 1

    SUM=28, VARIATIONS=27
    SUM=29, VARIATIONS=81
    SUM=30, VARIATIONS=183
    SUM=31, VARIATIONS=296
    SUM=32, VARIATIONS=356
    SUM=33, VARIATIONS=356
    SUM=34, VARIATIONS=296
    SUM=35, VARIATIONS=183
    SUM=36, VARIATIONS=81
    SUM=37, VARIATIONS=27

### Exponent 2

    SUM=255, VARIATIONS=9


# Without summed inner edges

## Solutions to edge-length 3

    SUM=9, VARIATIONS=1
    SUM=10, VARIATIONS=1
    SUM=11, VARIATIONS=1
    SUM=12, VARIATIONS=1

## Solutions to edge-length 4
    
    SUM=17, VARIATIONS=16
    SUM=19, VARIATIONS=32
    SUM=20, VARIATIONS=48
    SUM=21, VARIATIONS=32
    SUM=23, VARIATIONS=16

## Solutions to edge-length 5

    SUM=28, VARIATIONS=3888
    SUM=29, VARIATIONS=8208
    SUM=30, VARIATIONS=15336
    SUM=31, VARIATIONS=23328
    SUM=32, VARIATIONS=24840
    SUM=33, VARIATIONS=24840
    SUM=34, VARIATIONS=23328
    SUM=35, VARIATIONS=15336
    SUM=36, VARIATIONS=8208
    SUM=37, VARIATIONS=3888


In [ ]:
find_magic_triangles(3, exponent=1, start_offset=1, verbose=False)
find_magic_triangles(4, exponent=1, start_offset=1, verbose=False)
find_magic_triangles(5, exponent=1, start_offset=1, verbose=False)

In [ ]:
find_magic_triangles(4, start_offset=0)


      2
    1   3
    
        3
      2   4
    1   6  5

          4
        3   5
      2       6 
    1   9   8   7

            5
          4   6
        3       7
      2           8
    1   12  11  10  9

    L   N   P
    2   3   6
    3   6   720
    4   9   362,880
    5  12

In [ ]:
for i in range(10):
    print(i, math.factorial(i))

In [ ]:
PIL.ImageDraw.ImageDraw.text?